# Starting point

We'll start from the previous solution. You can click the button "Run all initialization cells" from the toolbar to run all the required cells.

## Tower class

In [ ]:
from typing import List

class Tower:
    def __init__(self) -> None:
        """Creates a tower"""
        self._discs: List[int] = []        

    def push(self, disc: int) -> None:
        self._discs.append(disc)

    def pop(self) -> int:
        return self._discs.pop()

    def __repr__(self) -> str:
        return repr(self._discs)
    
    def __contains__(self, disc: int) -> bool:
        return disc in self._discs

## HanoiTowers class

In [ ]:
from typing import Tuple, Iterator

class HanoiTowers:
    SOURCE: str = "SOURCE"
    TEMP: str = "TEMPORARY"
    DEST: str = "DESTINATION"
    def __init__(self, num_discs: int):
        self.source: Tower = Tower()
        self.destination: Tower = Tower()
        self.temporary: Tower = Tower()
        self.num_discs = num_discs
        
        for disc in range(num_discs, 0, -1):
            self.source.push(disc)

    def __iter__(self) -> Iterator[Tuple[str, Tower]]:
        yield from {
            self.SOURCE: self.source,
            self.TEMP: self.temporary,
            self.DEST: self.destination,
        }.items()
    
    def __len__(self) -> int:
        return 3

## HanoiPrinter class

In [ ]:
from typing import List
from dataclasses import dataclass

@dataclass
class HanoiPrinter:
    empty_level: str = "|"
    tower_base_fill: str = "="
    disc_fill: str = "X"
    
    def draw(self, towers: HanoiTowers) -> None:
        for name, tower in towers:
            title = f"{name} Tower"
            print(title)
            print("=" * len(title) + "\n")

            print(self.empty_level)
            for disc in range(1, towers.num_discs + 1):
                if disc in tower:
                    print(self.disc_fill * disc)
                else:
                    print(self.empty_level)
            print(self.tower_base_fill * towers.num_discs + "\n")


## HanoiSolver class

In [ ]:
class HanoiSolver:
    def _hanoi(self, source: Tower, destination: Tower, temporary: Tower, num_discs: int):
        if num_discs == 1:
            # Just move disc from source to destination
            destination.push(source.pop())
        else:
            # Move N - 1 discs from the source to the temporary tower.
            # This means we "swap" destination and temporary
            self._hanoi(
                source=source,
                destination=temporary,
                temporary=destination,
                num_discs=num_discs - 1,
            )
            # Move the remaining disc from the source to the destination
            self._hanoi(
                source=source,
                destination=destination,
                temporary=temporary,
                num_discs=1,
            )
            # Finally, move the N - 1 discs from the first step to the destination.
            # This means that we are swapping the source and the temporary towers.
            self._hanoi(
                source=temporary,
                destination=destination,
                temporary=source,
                num_discs=num_discs - 1,
            )
    def solve(self, towers: HanoiTowers):
        self._hanoi(
            source=towers.source,
            destination=towers.destination,
            temporary=towers.temporary,
            num_discs=towers.num_discs,
        )

## Hanoi class

In [ ]:
class Hanoi:
    def __init__(self, towers: HanoiTowers, printer: HanoiPrinter, solver: HanoiSolver):
        self.towers = towers
        self.printer = printer
        self.solver = solver

    def draw(self):
        self.printer.draw(towers=self.towers)
    
    def solve(self):
        self.solver.solve(towers=self.towers)

    def draw_and_solve(self):
        print("Initial State:\n")
        self.draw()
        self.solve()
        print("\n\nFinal State:\n")
        self.draw()
    
    @staticmethod
    def create(num_discs: int) -> "Hanoi":
        towers = HanoiTowers(num_discs=num_discs)
        printer = HanoiPrinter()
        solver = HanoiSolver()
        
        return Hanoi(towers=towers, printer=printer, solver=solver)

# O: Open/Closed principle

So, is this implementation following the open/closed principle? It doesn't seem so. For instance, the ```HanoiPrinter``` class is not *closed* for change or *open* for extension. This means that if we have different types of towers (e.g. if we have a tower for which we don't want to show the discs, or show them differently), we'll have to change the ```draw``` method every time.

So we will start by making the ```draw``` method closed to change, while still allowing different types of ```Tower``` classes to be added. We can achieve the same in different ways:

- Create a ```draw``` method directly in the ```Tower``` class, so we just need to call ```tower.draw()``` from our ```HanoiPrinter```. This would work, but it makes the Tower class also responsible for printing it, so it would violate the Single Responsibility Principle.

- Decide what are the parts we want to be able to customize (e.g. the empty level, tower base and disc fill), and move this to the ```Tower``` class.

We'll go for the second option, which is theoretically not 100% correct, but it's a good trade-off for our problem. In short, this decision will be good as long as we're sure that all the customization we want to have is only on these characters we just mentioned, but not in how we draw the tower itself.

## Change the Tower class

In [ ]:
from typing import List

class Tower:
    def __init__(
        self,
        empty_level: str = "|",
        tower_base_fill: str = "=",
        disc_fill: str = "X",
    ) -> None:
        """Creates an empty tower"""
        self.empty_level = empty_level
        self.tower_base_fill = tower_base_fill
        self.disc_fill = disc_fill
        self._discs = []        

    def push(self, disc: int) -> None:
        self._discs.append(disc)

    def pop(self) -> int:
        return self._discs.pop()

    def __repr__(self) -> str:
        return repr(self._discs)
    
    def __contains__(self, disc: int) -> bool:
        return disc in self._discs

## Adapt the HanoiPrinter class

Now, we'll update the ```HanoiPrinter``` class to use the characters from the towers.

In [ ]:
from typing import List
from dataclasses import dataclass

class HanoiPrinter:
    def draw(self, towers: HanoiTowers) -> None:
        for name, tower in towers:
            title = f"{name} Tower"
            print(title)
            print("=" * len(title) + "\n")

            print(tower.empty_level)
            for disc in range(1, towers.num_discs + 1):
                if disc in tower:
                    print(tower.disc_fill * disc)
                else:
                    print(tower.empty_level)
            print(tower.tower_base_fill * towers.num_discs + "\n")


## Adapt the HanoiTowers class

The change we introduced simplified the HanoiPrinter class, but it adds some extra complexity to the ```HanoiTowers``` if we want to be able to actually customize the different towers: we should inject them rather than create them from within the class. This is not necessarily bad, since it will reduce the coupling between the ```Tower``` and the ```HanoiTowers``` classes

In [ ]:
from typing import Tuple, Iterator

class HanoiTowers:
    SOURCE: str = "SOURCE"
    TEMP: str = "TEMPORARY"
    DEST: str = "DESTINATION"
    def __init__(self, num_discs: int, source: Tower, temporary: Tower, destination: Tower):
        self.num_discs = num_discs
        self.source: Tower = source
        self.destination: Tower = destination
        self.temporary: Tower = temporary
        
        for disc in range(num_discs, 0, -1):
            self.source.push(disc)

    def __iter__(self) -> Iterator[Tuple[str, Tower]]:
        yield from {
            self.SOURCE: self.source,
            self.TEMP: self.temporary,
            self.DEST: self.destination,
        }.items()
    
    def __len__(self) -> int:
        return 3
    
    @staticmethod
    def default_towers(num_discs: int) -> "HanoiTowers":
        source = Tower()
        destination = Tower()
        temporary = Tower()
        
        for disc in range(num_discs, 0, -1):
            source.push(disc)
        
        return HanoiTowers(
            num_discs=num_discs,
            source=source,
            temporary=temporary,
            destination=destination,
        )
        

## Other changes

Finally, we'll have to update our convenience method in the ```Hanoi``` class to reflect this. The fact that we need to change that class because we updated another class is an indication (or *code smell*) that something's not right. If you think of it, by having this shortcut, we're adding extra coupling between the classes, which is why we now need to change it. The alternative, though (not having the ```create``` method, would potentially require us to change the code in the different applications, so once again this is about a trade-off between how theoretically correct the code is and how easy it is to work with it.

In [ ]:
class Hanoi:
    def __init__(self, towers: HanoiTowers, printer: HanoiPrinter, solver: HanoiSolver):
        self.towers = towers
        self.printer = printer
        self.solver = solver

    def draw(self):
        self.printer.draw(towers=self.towers)
    
    def solve(self):
        self.solver.solve(towers=self.towers)

    def draw_and_solve(self):
        print("Initial State:\n")
        self.draw()
        self.solve()
        print("\n\nFinal State:\n")
        self.draw()
    
    @staticmethod
    def create(num_discs: int) -> "Hanoi":
        towers = HanoiTowers.default_towers(num_discs=num_discs)
        printer = HanoiPrinter()
        solver = HanoiSolver()
        
        return Hanoi(towers=towers, printer=printer, solver=solver)

In [ ]:
h = Hanoi.create(num_discs=2)
h.draw_and_solve()

This works exactly the same as before. The difference is that now we have the option to customize a lot more. Suppose we want to change customize the tower base and the disc fill for every tower, now we can do it like this:

In [ ]:
num_discs = 3
source = Tower(tower_base_fill="SS", disc_fill="o")
temp = Tower(tower_base_fill="T", disc_fill="d")
dest = Tower(tower_base_fill="DD", disc_fill="X")
towers = HanoiTowers(
    num_discs=num_discs,
    source=source,
    destination=dest,
    temporary=temp,
)
printer = HanoiPrinter()
solver = HanoiSolver()
h = Hanoi(towers=towers, printer=printer, solver=solver)
h.draw_and_solve()

# Final Code

Here's the final code for the solution proposed:

In [ ]:
# Tower class
from typing import List

class Tower:
    def __init__(
        self,
        empty_level: str = "|",
        tower_base_fill: str = "=",
        disc_fill: str = "X",
    ) -> None:
        """Creates an empty tower"""
        self.empty_level = empty_level
        self.tower_base_fill = tower_base_fill
        self.disc_fill = disc_fill
        self._discs = []        

    def push(self, disc: int) -> None:
        self._discs.append(disc)

    def pop(self) -> int:
        return self._discs.pop()

    def __repr__(self) -> str:
        return repr(self._discs)
    
    def __contains__(self, disc: int) -> bool:
        return disc in self._discs

# HanoiTowers class
from typing import Tuple, Iterator

from typing import Tuple, Iterator

class HanoiTowers:
    SOURCE: str = "SOURCE"
    TEMP: str = "TEMPORARY"
    DEST: str = "DESTINATION"
    def __init__(self, num_discs: int, source: Tower, temporary: Tower, destination: Tower):
        self.num_discs = num_discs
        self.source: Tower = source
        self.destination: Tower = destination
        self.temporary: Tower = temporary
        
        for disc in range(num_discs, 0, -1):
            self.source.push(disc)

    def __iter__(self) -> Iterator[Tuple[str, Tower]]:
        yield from {
            self.SOURCE: self.source,
            self.TEMP: self.temporary,
            self.DEST: self.destination,
        }.items()
    
    def __len__(self) -> int:
        return 3
    
    @staticmethod
    def default_towers(num_discs: int) -> "HanoiTowers":
        source = Tower()
        destination = Tower()
        temporary = Tower()
        
        for disc in range(num_discs, 0, -1):
            source.push(disc)
        
        return HanoiTowers(
            num_discs=num_discs,
            source=source,
            temporary=temporary,
            destination=destination,
        )

# HanoiPrinter class
from typing import List
from dataclasses import dataclass

class HanoiPrinter:
    def draw(self, towers: HanoiTowers) -> None:
        for name, tower in towers:
            title = f"{name} Tower"
            print(title)
            print("=" * len(title) + "\n")

            print(tower.empty_level)
            for disc in range(1, towers.num_discs + 1):
                if disc in tower:
                    print(tower.disc_fill * disc)
                else:
                    print(tower.empty_level)
            print(tower.tower_base_fill * towers.num_discs + "\n")


# HanoiSolver
class HanoiSolver:
    def _hanoi(self, source: Tower, destination: Tower, temporary: Tower, num_discs: int):
        if num_discs == 1:
            # Just move disc from source to destination
            destination.push(source.pop())
        else:
            # Move N - 1 discs from the source to the temporary tower.
            # This means we "swap" destination and temporary
            self._hanoi(
                source=source,
                destination=temporary,
                temporary=destination,
                num_discs=num_discs - 1,
            )
            # Move the remaining disc from the source to the destination
            self._hanoi(
                source=source,
                destination=destination,
                temporary=temporary,
                num_discs=1,
            )
            # Finally, move the N - 1 discs from the first step to the destination.
            # This means that we are swapping the source and the temporary towers.
            self._hanoi(
                source=temporary,
                destination=destination,
                temporary=source,
                num_discs=num_discs - 1,
            )
    def solve(self, towers: HanoiTowers):
        self._hanoi(
            source=towers.source,
            destination=towers.destination,
            temporary=towers.temporary,
            num_discs=towers.num_discs,
        )

# Hanoi class
class Hanoi:
    def __init__(self, towers: HanoiTowers, printer: HanoiPrinter, solver: HanoiSolver):
        self.towers = towers
        self.printer = printer
        self.solver = solver

    def draw(self):
        self.printer.draw(towers=self.towers)
    
    def solve(self):
        self.solver.solve(towers=self.towers)

    def draw_and_solve(self):
        print("Initial State:\n")
        self.draw()
        self.solve()
        print("\n\nFinal State:\n")
        self.draw()
    
    @staticmethod
    def create(num_discs: int) -> "Hanoi":
        towers = HanoiTowers.default_towers(num_discs=num_discs)
        printer = HanoiPrinter()
        solver = HanoiSolver()
        
        return Hanoi(towers=towers, printer=printer, solver=solver)